# Tarea 1 - Fraschini, Rovira - Grupo 2

Cargar el dataset usando la libreria pandas y guardarlo en una variable llamada dataset.

In [ ]:
import pandas
dataset = pandas.read_csv("../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")

## Parte 1 

Análisis de los datos y correlación.

### Tipos

Para comenzar con el análisis de los datos, obtendremos el tipo de cada una de las variables que lo conforman y la cantidad de valores no nulos ingresados en cada una de ellas.

In [ ]:
dataset.info()

Con esto, se puede observar que el dataset tiene 12 columnas y 1599 filas. De las 12 columnas, 11 son de tipo float64 y una de tipo int64. Además se puede ver que no hay ningún dato nulo ya que todas las columnas tienen 1599 datos no nulos que es el total de datos del dataset.

### Descripción del dataset

Luego, usaremos la funcion describe() sobre el dataset para obtener: cantidad de entradas del dataset (count), promedio de cada una de las variables (mean), desviacion estandar (std). Además incluye los 5 percentiles mas importantes que son: valor minimo (min); valor del primer cuartil, es decir que el 25% de los datos estan por debajo de este valor (25%); valor del segundo cuartil, el 50% de los datos esta por debajo de este valor y el otro 50% por encima (50%); valor del tercer cuartil, el 75% de los datos esta por debajo de este valor (75%); y por último el valor máximo de la variable (max) 

In [ ]:
dataset.describe(include='all')

### Correlación entre todas las variables 2 a 2.

La correlación se expresa en un coeficiente de correlación que varía entre -1 y +1. La correlación negativa es cuando un incremento en una variable causa un decremento en la otra. La correlación cero es que no hay ninguna relación entre las variaciones de las dos variables. La correlación de una variable con ella misma siempre es +1. Cuanto más cerca de 1 este el valor absoluto de la correlación entre de 2 variables, más estrechamente ligados estan los cambios de una con los de la otra. La correlación no responde a una relación de causa y efecto.
Para obtener la correlación entre todas las variabes 2 a 2, se invocará la funcion corr() sobre el dataset. 

In [ ]:
dataset.corr()

### Gráfica de Correlación 

Se usa la función heatmap de seaborn y matplotlib para renderizar los valores de correlación hallados anteriormente. 
Las celdas quedan coloreadas según su valor. 
Podemos ver como la diagonal es la parte más oscura del heatmap ya que es la correlación más alta porque es la de cada variable consigo misma.
Luego hay diferentes matices pero se observa que ninguna de las correlaciones supera el 0.7 de valor absoluto. Más aún, la correlación entre la calidad del vino que es la variable de estudio, y las demás variables nunca alcanza el 0.5.

In [ ]:
import seaborn
import matplotlib.pyplot as plot

cm = seaborn.color_palette("Spectral", as_cmap=True)
plot.figure(figsize=(15, 8))
seaborn.heatmap(dataset.corr(), annot = True, square=True, cmap=cm)
plot.title('Correlación')
plot.show()

## Parte 2

Se aplicará el método de regresión lineal para predecir la calidad del vino teniendo en cuenta los valores del resto de las variables. Para esto se separa el dataset en un conjunto de datos de entrada que contiene todos los features salvo la calidad del vino y otro conjunto de datos que sería la salida que solo contiene los valores de calidad del vino.
Se usará el 80% de los datos para entrenar el modelo y el otro 20% para testearlo.
La regresión lineal es un algoritmo de aprendizaje supervisado que estudia la mejor relación funcional para un conjunto de puntos. El mejor modelo es aquel que minimiza el error de Y = B0 + B1X. 


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import RFE
import numpy

linearReg = LinearRegression()

x = dataset.drop(['quality'],axis=1)
y = dataset['quality']

spl = 0.8
N = len(y)
sample = int(spl*N)

x_train, x_test, y_train, y_test = x[:sample], x[sample:], y[:sample], y[sample:]

linearReg.fit(x_train,y_train)

predictionsLin = linearReg.predict(x_test)
res = numpy.c_[predictionsLin, y_test]

# print(res)

plot.figure(figsize=(15,8))
plot.ylabel('Predicciones')
plot.xlabel('Valores de test')
plot.scatter(y_test,predictionsLin, alpha=0.3)

print('Mean Squared Error:', mean_squared_error(y_test, predictionsLin))

Aplicando regresión lineal al dataset obtenemos un error cuadrático medio de 0.43. Que es el promedio de los errores al cuadrado. Los errores son la diferencia entre las variables y de test y las predicciones realizadas por el algoritmo.

### Mecanismos de mejora

#### Eliminacion de ouliers

En primer lugar, intentaremos determinar que variables en el dataset tienen outliers, es decir, valores numéricamente distantes del resto de los datos, que puedan afectar la predicción que se realiza. Luego de determinar los valores a partir de los cuales una entrada se consideraría un outlier, eliminaremos esas entradas y se realizará de nuevo la regresión lineal con el nuevo dataset sin outliers.
A continuación se grafican todas las variables contra la calidad del vino para determinar los valores a partir de los cuales consideraremos que el dato es un oulier.

In [ ]:
dataOut = dataset ## usamos este nuevo dataset llamado dataOut para sacarle los outliers al original
seaborn.set(rc={'figure.figsize':(8,5)})
seaborn.boxplot(x=dataOut['quality'], y=dataOut['fixed acidity'])

Consideraremos que cualquier valor mayor a 11 de fixed acidity es un outlier

In [ ]:
seaborn.boxplot(x=dataOut['quality'], y=dataOut['volatile acidity'])

Consideraremos que cualquier valor mayor a 1.1 de volatile acidity es un outlier

In [ ]:
seaborn.boxplot(x=dataOut['quality'], y=dataOut['citric acid'])

Consideraremos que cualquier valor mayor a 0.6 de citric acid es un outlier

In [ ]:
seaborn.boxplot(x=dataOut['quality'], y=dataOut['residual sugar'])

Consideraremos que cualquier valor mayor a 3.5 de residual sugar es un outlier

In [ ]:
seaborn.boxplot(x=dataOut['quality'], y=dataOut['chlorides'])

Consideraremos que cualquier valor mayor a 0.16 de chlorides es un outlier

In [ ]:
seaborn.boxplot(x=dataOut['quality'], y=dataOut['free sulfur dioxide'])

Consideraremos que cualquier valor mayor a 25 de free sulfur dioxide es un outlier

In [ ]:
seaborn.boxplot(x=dataOut['quality'], y=dataOut['total sulfur dioxide'])

Consideraremos que cualquier valor mayor a 90 de total sulfur dioxide es un outlier

In [ ]:
seaborn.boxplot(x=dataOut['quality'], y=dataOut['density'])

Consideraremos que cualquier valor mayor a 1000 o menor a 0.994 de density es un outlier

In [ ]:
seaborn.boxplot(x=dataOut['quality'], y=dataOut['pH'])

Consideraremos que cualquier valor mayor a 3.6 o menor a 3.0 de pH es un outlier

In [ ]:
seaborn.boxplot(x=dataOut['quality'], y=dataOut['sulphates'])

Consideraremos que cualquier valor mayor a 0.88 de sulphates es un outlier

In [ ]:
seaborn.boxplot(x=dataOut['quality'], y=dataOut['alcohol'])

Consideraremos que cualquier valor mayor a 13 de alcohol es un outlier.
Eliminaremos todas las entradas con outliers y haremos de nuevo la predicción usando regresión lineal.

In [ ]:
dataOut = dataOut.drop(dataOut[dataOut['fixed acidity'] > 11].index)
dataOut = dataOut.drop(dataOut[dataOut['volatile acidity'] > 1.1].index)
dataOut = dataOut.drop(dataOut[dataOut['citric acid'] > 0.6].index)
dataOut = dataOut.drop(dataOut[dataOut['residual sugar'] > 3.5].index)
dataOut = dataOut.drop(dataOut[dataOut['chlorides'] > 0.16].index)
dataOut = dataOut.drop(dataOut[dataOut['free sulfur dioxide'] > 25].index)
dataOut = dataOut.drop(dataOut[dataOut['total sulfur dioxide'] > 90].index)
dataOut = dataOut.drop(dataOut[dataOut['density'] > 1000].index)
dataOut = dataOut.drop(dataOut[dataOut['density'] < 0.994].index)
dataOut = dataOut.drop(dataOut[dataOut['pH'] > 3.6].index)
dataOut = dataOut.drop(dataOut[dataOut['pH'] < 3.0].index)
dataOut = dataOut.drop(dataOut[dataOut['sulphates'] > 0.88].index)
dataOut = dataOut.drop(dataOut[dataOut['alcohol'] > 13].index)

x = dataOut.drop(['quality'],axis=1)
y = dataOut['quality']

spl = 0.8
N = len(y)
sample = int(spl*N)

x_train, x_test, y_train, y_test = x[:sample], x[sample:], y[:sample], y[sample:]

linearReg.fit(x_train,y_train)

predictionsOut = linearReg.predict(x_test)
res = numpy.c_[predictionsOut, y_test]

# print(res)

plot.ylabel('Predicciones')
plot.xlabel('Valores de test')
plot.scatter(y_test,predictionsOut, alpha=0.3)

print('Mean Squared Error:', mean_squared_error(y_test, predictionsOut))
dataOut.shape

Se puede observar que el dataset ahora tiene solo 800 entradas y antes eran 1599.
El error, que antes era de 0.43, ahora es de 0.39

#### RFE

Otro mecanismo de mejora es eliminar los features que no colaboran a la predicción. Para esto utilizaremos el RFE de sklearn que selecciona las columnas más relevantes en la predicción sobre el dataset original (es decir, el que tiene outliers).
Realizando pruebas, pudimos ver que el numero optimo de features era 7. 

In [ ]:
from sklearn.feature_selection import RFE

x = dataset.drop(['quality'],axis=1)
y = dataset['quality']

spl = 0.8
N = len(y)
sample = int(spl*N)

x_train, x_test, y_train, y_test = x[:sample], x[sample:], y[:sample], y[sample:]

linearReg = LinearRegression()
rfe = RFE(linearReg, n_features_to_select=7)

rfe.fit(x_train,y_train)

predictionsRFE = rfe.predict(x_test)
res = numpy.c_[predictionsRFE, y_test]

# print(res)

plot.ylabel('Predicciones')
plot.xlabel('Valores de test')
plot.scatter(y_test,predictionsRFE, alpha=0.3)

print('Mean Squared Error:', mean_squared_error(y_test, predictionsRFE))
             
isSelected = pandas.Series(rfe.support_,index = x_train.columns)
selected_features_rfe = isSelected[isSelected==True].index
selectedToShow = pandas.DataFrame(selected_features_rfe, 
             columns=['Varibales seleccionadas'])
display(selectedToShow)

Utilizando RFE sobre el dataset original el error es de 0.42.

#### RFE sobre el dataset sin outliers

Por último aplicaremos la eliminación de features al dataset sin outliers.

In [ ]:
x = dataOut.drop(['quality'],axis=1)
y = dataOut['quality']

spl = 0.8
N = len(y)
sample = int(spl*N)

x_train, x_test, y_train, y_test = x[:sample], x[sample:], y[:sample], y[sample:]

linearReg = LinearRegression()
rfe = RFE(linearReg, n_features_to_select=7)

rfe.fit(x_train,y_train)

predictionsOutRFE = rfe.predict(x_test)
res = numpy.c_[predictionsOutRFE, y_test]

# print(res)

plot.ylabel('Predicciones')
plot.xlabel('Valores de test')
plot.scatter(y_test,predictionsOutRFE, alpha=0.3)

print('Mean Squared Error:', mean_squared_error(y_test, predictionsOutRFE))
             
isSelected = pandas.Series(rfe.support_,index = x_train.columns)
selected_features_rfe = isSelected[isSelected==True].index
selectedToShow = pandas.DataFrame(selected_features_rfe, 
             columns=['Varibales seleccionadas'])
display(selectedToShow)

En esta última predicción se obtiene un error menor a todos los anteriores que se obtuvieron usando regresión lineal. Aunque tanto eliminar los outliers como eliminar lso features que no colaboran a la predicción ayudan. Se observa que eliminar los outliers tiene un mayor impacto y que, si se aplican ambos mecanismos en simultáneo, el error es el menor obtenido

## Parte 3

Se clasificará la calidad del vino como buena o mala. 
Luego se utilizará el método de regresión logística y el de Naive Bayes para realizar las predicciones y se compararán los resultados usando algunas métricas.

In [ ]:
dataset['quality'].value_counts()

#### Umbral

Utilizaremos como valor de corte para la clasificación el 6. 
Todas las entradas cuyo valor de calidad sea menor o igual a 6 serán etiquetadas como vino malo. Las que tengan una calidad mayor a 6 serán etiquetadas como vino bueno.

Agregamos un 0 y un 1 delante de las categorias porque sino la grafica de roc se da vuelta por un tema de orden alfabético.

In [ ]:
dataset['quality'] = pandas.cut(dataset['quality'], bins = (2, 6, 8), labels = ['0malo', '1bueno'])

dataset.head(10)

#### Método de regresión logística



In [ ]:
from sklearn.linear_model import LogisticRegression

x = dataset.drop(['quality'],axis=1)
y = dataset['quality']

spl = 0.8
N = len(y)
sample = int(spl*N)

x_train, x_test, y_train, y_test = x[:sample], x[sample:], y[:sample], y[sample:]

logRegression = LogisticRegression(solver='lbfgs', max_iter=10000)
logRegression.fit(x_train, y_train)
predictionsLR = logRegression.predict(x_test)

#### Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

x = dataset.drop(['quality'],axis=1)
y = dataset['quality']

spl = 0.8
N = len(y)
sample = int(spl*N)

x_train, x_test, y_train, y_test = x[:sample], x[sample:], y[:sample], y[sample:]

naiveBayes=GaussianNB()
naiveBayes.fit(x_train,y_train)
predictionsNB=naiveBayes.predict(x_test)

In [ ]:
import seaborn
from sklearn.metrics import confusion_matrix

mat = confusion_matrix(y_test, predictionsLR)
seaborn.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False, cmap=cm)
plot.xlabel('Test')
plot.ylabel('Prediccion Regresion Logistica')

In [ ]:
matNB = confusion_matrix(y_test, predictionsNB)
seaborn.heatmap(matNB.T, square=True, annot=True, fmt='d', cbar=False, cmap=cm)
plot.xlabel('Test')
plot.ylabel('Prediccion Naive Bayes')

#### Matriz de confusión

Analizando las matrices de confusión que se desprenden de las predicciones de cada uno de los métodos, podemos ver que usando regresión logística los resultados son mejores ya que acierta 296 veces y no lo hace solo en 24 casos. 
Por otro lado, el algoritmo de Naive Bayes acierta 283 veces y no lo hace 37 veces.

#### Curva ROC

A continuacion graficaremos la ROC para ambos métodos. 
fpr es la razon de falsos positivos y tpr es la razon de verdaderos positivos.
Si la ROC grafica un rectangulo, la prediccion es perfecta y el modelo es capaz de distinguir sin error entre un buen vino y un mal vino.

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

probasLR = logRegression.predict_proba(x_test)
fpr, tpr, thresholds = roc_curve(y_test, probasLR[:, 1], pos_label='1bueno')

plot.plot([0,1], [0,1], 'k-.')
plot.plot(fpr, tpr, label = 'LR')
plot.xlabel('fpr')
plot.ylabel('tpr')
plot.title('ROC para regresion logistica')
plot.show()


In [ ]:
probasNB = naiveBayes.predict_proba(x_test)
fpr, tpr, thresholds = roc_curve(y_test, probasNB[:, 1], pos_label='1bueno')

plot.plot([0,1],[0,1],'k-.')
plot.plot(fpr, tpr, label='NB') 
plot.xlabel('fpr')
plot.ylabel('tpr')
plot.title('Curva ROC para Naive Bayes')
plot.show()

#### Area bajo la curva roc

Lo importante es el area bajo la curva que analizaremos a continuacion. 
El modelo es perfecto si el area es 1. Y cuanto más cerca de 1 este ese numero, mejor es la predicción aunque no sea absolutamnete perfecta.

In [ ]:
print(roc_auc_score(y_test, probasLR[:, 1]))
print(roc_auc_score(y_test, probasNB[:, 1]))

Aqui se observa que el modelo de Naive Bayes arroja un resultado mejor que el de regresion logistica cuando se trata del area bajo la curva roc.

#### Reporte de clasificacion

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

print (classification_report(y_test,predictionsLR))

In [ ]:
print (classification_report(y_test,predictionsNB))

El report de clasificacion lanza varias metricas.
En primer lugar, la precision indica que tan preciso es el modelo para la prediccion de positivos y de negativos.
El recall calcula cuantos de los verdaderos positivos (o negativos) son efectivamente capturados por el modelo.
El f1 score es una media armonica entre la precision y el recall.
En el promedio ponderado de estas metricas, podemos ver que funciona mejor la regresion logistica ya que tiene mejores valores en recall y en f1 score que naive bayes y tiene la misma precision.
En el promedio sin ponderar, el modelo naive bayes parece ser mejor en recall y f1 score mientras que el de regresion logistica es mejor en la precision.

#### Accuracy

In [ ]:
print (accuracy_score(y_test,predictionsLR))
print (accuracy_score(y_test,predictionsNB))

El accuracy es la cantidad de resultados verdaderos (true positive and true negative) sobre el total de los casos examinados.
En esta métrica es bastante mejor e. modelo de regresion logistica ya que tiene un accuracy del 93% frente a un 88% que arroja el modelo de Naive Bayes

#### Feature relevance

Técnica que asigna un valor a los features de entrada en función a la utilidad que tienen para predecir la calidad del vino, en este caso.
El permutation importance permite ver cuanto cambian los resultados del modelo si se cambian aleatoriamente de posición los valores de una columna. Si el modelo cambia mucho, es porque esa columna era importante para la predicción.
Además, el coef_ en la regresión logística es el coeficiente de cada uno de los features en la función de decisión.

In [ ]:
from sklearn.inspection import permutation_importance

impsLR = permutation_importance(logRegression, x_test, y_test)
importancesLR = impsLR.importances_mean
stdLR = impsLR.importances_std
indicesLR = numpy.argsort(importancesLR)[::-1]

impsNB = permutation_importance(naiveBayes, x_test, y_test)
importancesNB = impsNB.importances_mean
stdNB = impsNB.importances_std
indicesNB = numpy.argsort(importancesNB)[::-1]

importanceLR = pandas.DataFrame(logRegression.coef_[0], 
             x.columns, 
             columns=['coeficiente LR'])\
            .sort_values(by='coeficiente LR', ascending=False)

importancePermLR = pandas.DataFrame(importancesLR[indicesLR], 
             x.columns[indicesLR], 
             columns=['coeficiente permutación LR'])\
            .sort_values(by='coeficiente permutación LR', ascending=False)

importanceNB = pandas.DataFrame(importancesNB[indicesNB], 
             x.columns[indicesNB], 
             columns=['coeficiente permutación NB'])\
            .sort_values(by='coeficiente permutación NB', ascending=False)

display(importanceLR)
display(importancePermLR)
display(importanceNB)

## Parte 4

Bases teóricas del algoritmo k-NN (k-nearest neighbors) y aplicacion al dataset.

### Bases teóricas

El algoritmo del k-nearest neighbors es un algoritmo de clasificación que toma un montón de puntos marcados y los utiliza para aprender a etiquetar otros puntos. Este algoritmo clasifica los casos nuevos basados en su similitud con otros casos ya presentes en el conjunto de datos de entrenamiento. Los puntos de datos que están cerca entre sí se dice que son “vecinos”. El algoritmo se basa en este paradigma: Casos similares con las mismas etiquetas de clase están cerca el uno al otro.
La distancia entre dos casos es una medida de su disimilitud. Existen diferentes maneras de calcular la similitud, la distancia o la disimilitud de dos puntos de datos. El más común es la distancia de Euclidiana.
El k lo elige el investigador dependiendo del conjunto de datos e indica la cantidad de vecinos que se evaluarán. La exactitud del algoritmo depende del k elegido. Si se elige un k = 1 solo se considera el vecino más cercano de todos.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

x = dataset.drop(['quality'],axis=1)
y = dataset['quality']

spl = 0.8
N = len(y)
sample = int(spl*N)

x_train, x_test, y_train, y_test = x[:sample], x[sample:], y[:sample], y[sample:]

knn = KNeighborsClassifier()
knn.fit(x_train,y_train)
predictionsKNN=knn.predict(x_test)
print(classification_report(y_test, predictionsKNN))

In [ ]:
pandas.DataFrame({'models': ["Logistic regression","Naive Bayes","KNN"],
                           'accuracies': [accuracy_score(y_test,predictionsLR),accuracy_score(y_test,predictionsNB),accuracy_score(y_test,predictionsKNN)]})

### Comparación con parte 3 

Se puede observar que el algoritmo de knn tiene una mejor accuracy que el de naive bayes pero no que el de regresion logistica.

## Parte 5 

Luego de realizar el trabajo, se pudieron sacar varias conclusiones.
En primer lugar, la importancia de contar con un buen dataset, con muchos datos para evaluar pero lo más libre de outliers presentes en sus muestras ya que se vio que estos valores atípicos afectan notoriamente las predicciones.
Además, no siempre lo mejor es tener la mayor cantidad de features posibles, sino las que más colaboren con la predicción que se desea realizar. Algunas features del dataset es preferible dejarlas fuera del conjunto de datos que alimenta el modelo porque puede que lo perjudiquen.
También se observó que la primera predicción puede no ser la más acertada y se debe intentar mejorar los datos de entrada del modelo para mejorar las predicciones y disminuir el error lo más posible.
También se observo que no alcanza con evaluar los métodos con una sola métrica ya que muchas veces un método arroja mejores resultados en cierta métrica pero peores en otra y entonces la elección del método debe depender del objetivo que se persiga.
Se aprecia, además, lo sencillo que es realizar predicciones sobre un conjunto de datos determinado si se cuenta con las herramientas adecuadas.
En conclusión, se analizaron los datos con éxito y se logró aprender a trabajar con cada uno de los modelos vistos en el curso de manera práctica.